# Entraînement du modèle de classification des prunes africaines

Ce notebook utilise les fonctions existantes dans le dépôt pour entraîner le modèle de classification des prunes africaines en utilisant le jeu de données Kaggle "African Plums Dataset".

## 1. Configuration de l'environnement pour Google Colab

Commençons par cloner le dépôt GitHub et configurer l'environnement.

In [ ]:
# Vérifier si nous sommes dans Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f"Exécution dans Google Colab: {IN_COLAB}")

if IN_COLAB:
    # Cloner le dépôt GitHub
    !git clone https://github.com/CodeStorm-mbe/african-plums-classifier.git
    %cd african-plums-classifier
    
    # Installer les dépendances requises
    !pip install -r requirements.txt
    !pip install kaggle

## 2. Monter Google Drive pour la persistance des données

Pour conserver les données et les modèles entre les différents notebooks, nous allons utiliser Google Drive comme stockage persistant.

In [ ]:
# Monter Google Drive si nous sommes dans Colab
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Définir les chemins dans Google Drive
    DRIVE_PROJECT_DIR = "/content/drive/MyDrive/african-plums-classifier"
    DRIVE_DATA_DIR = f"{DRIVE_PROJECT_DIR}/data"
    DRIVE_KAGGLE_DIR = f"{DRIVE_DATA_DIR}/kaggle"
    DRIVE_RAW_DATA_DIR = f"{DRIVE_DATA_DIR}/raw"
    DRIVE_PLUM_DATA_DIR = f"{DRIVE_RAW_DATA_DIR}/plums"
    DRIVE_NON_PLUM_DATA_DIR = f"{DRIVE_RAW_DATA_DIR}/non_plums"
    DRIVE_MODELS_DIR = f"{DRIVE_PROJECT_DIR}/models"
    
    # Vérifier si les répertoires existent, sinon les créer
    !mkdir -p {DRIVE_PROJECT_DIR}
    !mkdir -p {DRIVE_DATA_DIR}
    !mkdir -p {DRIVE_KAGGLE_DIR}
    !mkdir -p {DRIVE_RAW_DATA_DIR}
    !mkdir -p {DRIVE_PLUM_DATA_DIR}
    !mkdir -p {DRIVE_NON_PLUM_DATA_DIR}
    !mkdir -p {DRIVE_MODELS_DIR}
    
    print(f"Google Drive monté et répertoires créés dans {DRIVE_PROJECT_DIR}")

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random
import json
import time
import shutil

# Ajouter le répertoire courant au chemin pour pouvoir importer nos modules
if IN_COLAB:
    # Dans Colab, nous sommes déjà dans le répertoire du projet
    if "/content/african-plums-classifier" not in sys.path:
        sys.path.append("/content/african-plums-classifier")
else:
    # En local, ajouter le répertoire parent
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

# Importer nos modules personnalisés
from data.data_preprocessing import load_and_prepare_two_stage_data
from models.model_architecture import get_model, TwoStageModel
from scripts.train_two_stage import train_model, evaluate_model, plot_training_history

# Définir les chemins des données
if IN_COLAB:
    # Utiliser les chemins dans Google Drive pour la persistance
    DATA_ROOT = DRIVE_RAW_DATA_DIR
    KAGGLE_DIR = DRIVE_KAGGLE_DIR
    MODELS_DIR = DRIVE_MODELS_DIR
    
    # Créer également des liens symboliques pour faciliter l'accès depuis le code existant
    LOCAL_DATA_ROOT = "data/raw"
    LOCAL_KAGGLE_DIR = "data/kaggle"
    LOCAL_MODELS_DIR = "models/saved"
    
    # Créer les répertoires locaux s'ils n'existent pas
    !mkdir -p {LOCAL_DATA_ROOT}
    !mkdir -p {LOCAL_KAGGLE_DIR}
    !mkdir -p {LOCAL_MODELS_DIR}
    
    # Créer des liens symboliques si nécessaire
    if not os.path.exists(LOCAL_DATA_ROOT) or not os.path.islink(LOCAL_DATA_ROOT):
        !rm -rf {LOCAL_DATA_ROOT}
        !ln -s {DATA_ROOT} {LOCAL_DATA_ROOT}
    
    if not os.path.exists(LOCAL_KAGGLE_DIR) or not os.path.islink(LOCAL_KAGGLE_DIR):
        !rm -rf {LOCAL_KAGGLE_DIR}
        !ln -s {KAGGLE_DIR} {LOCAL_KAGGLE_DIR}
        
    if not os.path.exists(LOCAL_MODELS_DIR) or not os.path.islink(LOCAL_MODELS_DIR):
        !rm -rf {LOCAL_MODELS_DIR}
        !ln -s {MODELS_DIR} {LOCAL_MODELS_DIR}
else:
    # En local
    DATA_ROOT = "../data/raw"
    KAGGLE_DIR = "../data/kaggle"
    MODELS_DIR = "../models/saved"

PLUM_DATA_DIR = os.path.join(DATA_ROOT, "plums")  # Sous-dossier pour les prunes
NON_PLUM_DATA_DIR = os.path.join(DATA_ROOT, "non_plums")  # Sous-dossier pour les non-prunes

# Créer les répertoires s'ils n'existent pas
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(PLUM_DATA_DIR, exist_ok=True)
os.makedirs(NON_PLUM_DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(KAGGLE_DIR, exist_ok=True)

# Définir les paramètres d'entraînement
BATCH_SIZE = 32
IMG_SIZE = 224
NUM_WORKERS = 2 if IN_COLAB else 4  # Réduire le nombre de workers dans Colab
LEARNING_RATE = 0.001
NUM_EPOCHS = 10 if IN_COLAB else 25  # Réduire le nombre d'époques dans Colab pour accélérer
EARLY_STOPPING_PATIENCE = 3 if IN_COLAB else 7  # Réduire la patience dans Colab
RANDOM_SEED = 42

# Fixer les seeds pour la reproductibilité
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Déterminer le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

# Afficher les chemins des données
print(f"\nChemins des données:")
print(f"DATA_ROOT: {DATA_ROOT}")
print(f"KAGGLE_DIR: {KAGGLE_DIR}")
print(f"PLUM_DATA_DIR: {PLUM_DATA_DIR}")
print(f"NON_PLUM_DATA_DIR: {NON_PLUM_DATA_DIR}")
print(f"MODELS_DIR: {MODELS_DIR}")

## 3. Vérification des données préparées dans Google Drive

Vérifions si les données ont déjà été préparées dans le notebook précédent et sont disponibles dans Google Drive.

In [ ]:
# Vérifier si les données ont été préparées dans le notebook précédent
def check_data_preparation():
    """Vérifie si les données ont été préparées dans le notebook précédent."""
    if IN_COLAB:
        # Vérifier si le fichier d'informations existe dans Google Drive
        data_prep_info_path = f"{DRIVE_PROJECT_DIR}/data_prep_info.json"
        if os.path.exists(data_prep_info_path):
            try:
                with open(data_prep_info_path, 'r') as f:
                    data_prep_info = json.load(f)
                
                if data_prep_info.get('data_prepared', False):
                    print(f"Les données ont été préparées le {data_prep_info.get('date_prepared', 'date inconnue')}.")
                    return data_prep_info
            except Exception as e:
                print(f"Erreur lors de la lecture du fichier d'informations: {e}")
    
    # Vérifier si les répertoires de données existent et contiennent des images
    plum_classes = [d for d in os.listdir(PLUM_DATA_DIR) if os.path.isdir(os.path.join(PLUM_DATA_DIR, d))]
    if not plum_classes:
        print(f"Aucune classe de prune trouvée dans {PLUM_DATA_DIR}.")
        return None
    
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    if not os.path.exists(non_plum_dir):
        print(f"Le répertoire {non_plum_dir} n'existe pas.")
        return None
    
    # Vérifier si les répertoires contiennent des images
    has_plum_images = False
    for cls in plum_classes:
        cls_dir = os.path.join(PLUM_DATA_DIR, cls)
        images = [f for f in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, f))]
        if images:
            has_plum_images = True
            break
    
    has_non_plum_images = False
    images = [f for f in os.listdir(non_plum_dir) if os.path.isfile(os.path.join(non_plum_dir, f))]
    if images:
        has_non_plum_images = True
    
    if has_plum_images and has_non_plum_images:
        print("Les données semblent être préparées (images trouvées dans les répertoires).")
        return {
            'data_prepared': True,
            'plum_classes': plum_classes,
            'detection_class_names': ['plum', 'non_plum'],
            'classification_class_names': plum_classes
        }
    else:
        print("Les données ne semblent pas être complètement préparées.")
        return None

# Vérifier si les données ont été préparées
data_prep_info = check_data_preparation()

if data_prep_info:
    print("\nInformations sur les données préparées:")
    for key, value in data_prep_info.items():
        print(f"  - {key}: {value}")
else:
    print("\nVeuillez d'abord exécuter le notebook de préparation des données.")

## 4. Chargement des données pour l'entraînement

Chargeons les données préparées pour l'entraînement du modèle.

In [ ]:
# Charger les données pour l'entraînement
def load_data_for_training():
    """Charge les données pour l'entraînement du modèle."""
    try:
        # Charger et préparer les données pour les deux étapes
        print("Chargement des données pour les deux étapes...")
        (detection_train_loader, detection_val_loader, detection_test_loader, detection_class_names), \
        (classification_train_loader, classification_val_loader, classification_test_loader, classification_class_names) = \
            load_and_prepare_two_stage_data(
                PLUM_DATA_DIR, 
                NON_PLUM_DATA_DIR,
                batch_size=BATCH_SIZE, 
                img_size=IMG_SIZE,
                num_workers=NUM_WORKERS
            )
        
        print(f"Classes de détection: {detection_class_names}")
        print(f"Classes de classification: {classification_class_names}")
        
        # Afficher les tailles des datasets
        print(f"\nTailles des datasets de détection:")
        print(f"  - Entraînement: {len(detection_train_loader.dataset)} images")
        print(f"  - Validation: {len(detection_val_loader.dataset)} images")
        print(f"  - Test: {len(detection_test_loader.dataset)} images")
        
        print(f"\nTailles des datasets de classification:")
        print(f"  - Entraînement: {len(classification_train_loader.dataset)} images")
        print(f"  - Validation: {len(classification_val_loader.dataset)} images")
        print(f"  - Test: {len(classification_test_loader.dataset)} images")
        
        return {
            'detection': {
                'train_loader': detection_train_loader,
                'val_loader': detection_val_loader,
                'test_loader': detection_test_loader,
                'class_names': detection_class_names
            },
            'classification': {
                'train_loader': classification_train_loader,
                'val_loader': classification_val_loader,
                'test_loader': classification_test_loader,
                'class_names': classification_class_names
            }
        }
    except Exception as e:
        print(f"Erreur lors du chargement des données: {e}")
        return None

# Charger les données si elles ont été préparées
if data_prep_info and data_prep_info.get('data_prepared', False):
    data_loaders = load_data_for_training()
    
    if data_loaders:
        detection_class_names = data_loaders['detection']['class_names']
        classification_class_names = data_loaders['classification']['class_names']
        
        detection_train_loader = data_loaders['detection']['train_loader']
        detection_val_loader = data_loaders['detection']['val_loader']
        detection_test_loader = data_loaders['detection']['test_loader']
        
        classification_train_loader = data_loaders['classification']['train_loader']
        classification_val_loader = data_loaders['classification']['val_loader']
        classification_test_loader = data_loaders['classification']['test_loader']
        
        print("\nDonnées chargées avec succès pour l'entraînement.")
    else:
        print("\nErreur lors du chargement des données. Veuillez vérifier les données préparées.")
else:
    print("\nVeuillez d'abord exécuter le notebook de préparation des données.")

## 5. Définition des modèles

Définissons les modèles pour la détection et la classification.

In [ ]:
# Définir les modèles
def create_models():
    """Crée les modèles pour la détection et la classification."""
    if 'detection_class_names' not in locals() or 'classification_class_names' not in locals():
        print("Les noms des classes ne sont pas définis. Veuillez d'abord charger les données.")
        return None, None
    
    try:
        # Modèle de détection (prune vs non-prune)
        print("Création du modèle de détection...")
        detection_model = get_model(
            model_name="standard",
            num_classes=len(detection_class_names),
            base_model="resnet18",
            pretrained=True
        )
        detection_model = detection_model.to(device)
        
        # Modèle de classification (types de prunes)
        print("Création du modèle de classification...")
        classification_model = get_model(
            model_name="standard",
            num_classes=len(classification_class_names),
            base_model="resnet34",
            pretrained=True
        )
        classification_model = classification_model.to(device)
        
        return detection_model, classification_model
    except Exception as e:
        print(f"Erreur lors de la création des modèles: {e}")
        return None, None

# Créer les modèles si les données ont été chargées
if 'detection_class_names' in locals() and 'classification_class_names' in locals():
    detection_model, classification_model = create_models()
    
    if detection_model is not None and classification_model is not None:
        print("\nModèles créés avec succès.")
        print(f"Modèle de détection: {detection_model.__class__.__name__}")
        print(f"Modèle de classification: {classification_model.__class__.__name__}")
    else:
        print("\nErreur lors de la création des modèles.")
else:
    print("\nVeuillez d'abord charger les données.")

## 6. Entraînement du modèle de détection

Entraînons d'abord le modèle de détection pour distinguer les prunes des non-prunes.

In [ ]:
# Entraîner le modèle de détection
def train_detection_model():
    """Entraîne le modèle de détection."""
    if 'detection_model' not in locals() or detection_model is None:
        print("Le modèle de détection n'est pas défini. Veuillez d'abord créer les modèles.")
        return None
    
    if 'detection_train_loader' not in locals() or 'detection_val_loader' not in locals():
        print("Les données d'entraînement ne sont pas définies. Veuillez d'abord charger les données.")
        return None
    
    try:
        # Définir l'optimiseur et le scheduler
        optimizer = optim.Adam(detection_model.parameters(), lr=LEARNING_RATE)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
        criterion = nn.CrossEntropyLoss()
        
        # Entraîner le modèle
        print("Entraînement du modèle de détection...")
        detection_history = train_model(
            model=detection_model,
            train_loader=detection_train_loader,
            val_loader=detection_val_loader,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            num_epochs=NUM_EPOCHS,
            device=device,
            early_stopping_patience=EARLY_STOPPING_PATIENCE,
            model_save_path=os.path.join(MODELS_DIR, 'detection_best_acc.pth')
        )
        
        # Tracer l'historique d'entraînement
        plot_training_history(detection_history)
        
        # Sauvegarder le graphique dans Google Drive si nous sommes dans Colab
        if IN_COLAB and os.path.exists('training_history.png'):
            detection_history_img_path = f"{DRIVE_PROJECT_DIR}/detection_training_history.png"
            shutil.copy('training_history.png', detection_history_img_path)
            print(f"Graphique d'historique d'entraînement sauvegardé dans Google Drive: {detection_history_img_path}")
        
        return detection_history
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle de détection: {e}")
        return None

# Entraîner le modèle de détection si les modèles ont été créés
if 'detection_model' in locals() and detection_model is not None and 'detection_train_loader' in locals():
    detection_history = train_detection_model()
    
    if detection_history is not None:
        print("\nModèle de détection entraîné avec succès.")
    else:
        print("\nErreur lors de l'entraînement du modèle de détection.")
else:
    print("\nVeuillez d'abord créer les modèles et charger les données.")

## 7. Évaluation du modèle de détection

Évaluons les performances du modèle de détection sur l'ensemble de test.

In [ ]:
# Évaluer le modèle de détection
def evaluate_detection_model():
    """Évalue le modèle de détection sur l'ensemble de test."""
    if 'detection_model' not in locals() or detection_model is None:
        print("Le modèle de détection n'est pas défini. Veuillez d'abord créer les modèles.")
        return None
    
    if 'detection_test_loader' not in locals():
        print("Les données de test ne sont pas définies. Veuillez d'abord charger les données.")
        return None
    
    try:
        # Charger le meilleur modèle
        best_model_path = os.path.join(MODELS_DIR, 'detection_best_acc.pth')
        if os.path.exists(best_model_path):
            detection_model.load_state_dict(torch.load(best_model_path, map_location=device))
            print(f"Meilleur modèle de détection chargé depuis {best_model_path}")
        
        # Évaluer le modèle
        print("Évaluation du modèle de détection sur l'ensemble de test...")
        criterion = nn.CrossEntropyLoss()
        test_loss, test_acc, test_f1, confusion_mat = evaluate_model(
            model=detection_model,
            test_loader=detection_test_loader,
            criterion=criterion,
            device=device,
            class_names=detection_class_names
        )
        
        print(f"\nRésultats sur l'ensemble de test:")
        print(f"  - Perte: {test_loss:.4f}")
        print(f"  - Précision: {test_acc:.4f}")
        print(f"  - Score F1: {test_f1:.4f}")
        
        return {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1_score': test_f1,
            'confusion_matrix': confusion_mat
        }
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle de détection: {e}")
        return None

# Évaluer le modèle de détection si le modèle a été entraîné
if 'detection_model' in locals() and detection_model is not None and 'detection_test_loader' in locals():
    detection_results = evaluate_detection_model()
    
    if detection_results is not None:
        print("\nModèle de détection évalué avec succès.")
    else:
        print("\nErreur lors de l'évaluation du modèle de détection.")
else:
    print("\nVeuillez d'abord entraîner le modèle de détection.")

## 8. Entraînement du modèle de classification

Entraînons maintenant le modèle de classification pour distinguer les différents types de prunes.

In [ ]:
# Entraîner le modèle de classification
def train_classification_model():
    """Entraîne le modèle de classification."""
    if 'classification_model' not in locals() or classification_model is None:
        print("Le modèle de classification n'est pas défini. Veuillez d'abord créer les modèles.")
        return None
    
    if 'classification_train_loader' not in locals() or 'classification_val_loader' not in locals():
        print("Les données d'entraînement ne sont pas définies. Veuillez d'abord charger les données.")
        return None
    
    try:
        # Définir l'optimiseur et le scheduler
        optimizer = optim.Adam(classification_model.parameters(), lr=LEARNING_RATE)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
        criterion = nn.CrossEntropyLoss()
        
        # Entraîner le modèle
        print("Entraînement du modèle de classification...")
        classification_history = train_model(
            model=classification_model,
            train_loader=classification_train_loader,
            val_loader=classification_val_loader,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            num_epochs=NUM_EPOCHS,
            device=device,
            early_stopping_patience=EARLY_STOPPING_PATIENCE,
            model_save_path=os.path.join(MODELS_DIR, 'classification_best_acc.pth')
        )
        
        # Tracer l'historique d'entraînement
        plot_training_history(classification_history)
        
        # Sauvegarder le graphique dans Google Drive si nous sommes dans Colab
        if IN_COLAB and os.path.exists('training_history.png'):
            classification_history_img_path = f"{DRIVE_PROJECT_DIR}/classification_training_history.png"
            shutil.copy('training_history.png', classification_history_img_path)
            print(f"Graphique d'historique d'entraînement sauvegardé dans Google Drive: {classification_history_img_path}")
        
        return classification_history
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle de classification: {e}")
        return None

# Entraîner le modèle de classification si les modèles ont été créés
if 'classification_model' in locals() and classification_model is not None and 'classification_train_loader' in locals():
    classification_history = train_classification_model()
    
    if classification_history is not None:
        print("\nModèle de classification entraîné avec succès.")
    else:
        print("\nErreur lors de l'entraînement du modèle de classification.")
else:
    print("\nVeuillez d'abord créer les modèles et charger les données.")

## 9. Évaluation du modèle de classification

Évaluons les performances du modèle de classification sur l'ensemble de test.

In [ ]:
# Évaluer le modèle de classification
def evaluate_classification_model():
    """Évalue le modèle de classification sur l'ensemble de test."""
    if 'classification_model' not in locals() or classification_model is None:
        print("Le modèle de classification n'est pas défini. Veuillez d'abord créer les modèles.")
        return None
    
    if 'classification_test_loader' not in locals():
        print("Les données de test ne sont pas définies. Veuillez d'abord charger les données.")
        return None
    
    try:
        # Charger le meilleur modèle
        best_model_path = os.path.join(MODELS_DIR, 'classification_best_acc.pth')
        if os.path.exists(best_model_path):
            classification_model.load_state_dict(torch.load(best_model_path, map_location=device))
            print(f"Meilleur modèle de classification chargé depuis {best_model_path}")
        
        # Évaluer le modèle
        print("Évaluation du modèle de classification sur l'ensemble de test...")
        criterion = nn.CrossEntropyLoss()
        test_loss, test_acc, test_f1, confusion_mat = evaluate_model(
            model=classification_model,
            test_loader=classification_test_loader,
            criterion=criterion,
            device=device,
            class_names=classification_class_names
        )
        
        print(f"\nRésultats sur l'ensemble de test:")
        print(f"  - Perte: {test_loss:.4f}")
        print(f"  - Précision: {test_acc:.4f}")
        print(f"  - Score F1: {test_f1:.4f}")
        
        return {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1_score': test_f1,
            'confusion_matrix': confusion_mat
        }
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle de classification: {e}")
        return None

# Évaluer le modèle de classification si le modèle a été entraîné
if 'classification_model' in locals() and classification_model is not None and 'classification_test_loader' in locals():
    classification_results = evaluate_classification_model()
    
    if classification_results is not None:
        print("\nModèle de classification évalué avec succès.")
    else:
        print("\nErreur lors de l'évaluation du modèle de classification.")
else:
    print("\nVeuillez d'abord entraîner le modèle de classification.")

## 10. Création et sauvegarde du modèle à deux étapes

Créons et sauvegardons le modèle à deux étapes qui combine le modèle de détection et le modèle de classification.

In [ ]:
# Créer et sauvegarder le modèle à deux étapes
def create_and_save_two_stage_model():
    """Crée et sauvegarde le modèle à deux étapes."""
    if 'detection_model' not in locals() or detection_model is None or 'classification_model' not in locals() or classification_model is None:
        print("Les modèles de détection et de classification ne sont pas définis. Veuillez d'abord créer les modèles.")
        return None
    
    try:
        # Charger les meilleurs modèles
        detection_best_path = os.path.join(MODELS_DIR, 'detection_best_acc.pth')
        classification_best_path = os.path.join(MODELS_DIR, 'classification_best_acc.pth')
        
        if os.path.exists(detection_best_path) and os.path.exists(classification_best_path):
            detection_model.load_state_dict(torch.load(detection_best_path, map_location=device))
            classification_model.load_state_dict(torch.load(classification_best_path, map_location=device))
            print("Meilleurs modèles chargés avec succès.")
        else:
            print("Les fichiers des meilleurs modèles n'existent pas. Utilisation des modèles actuels.")
        
        # Créer le modèle à deux étapes
        print("Création du modèle à deux étapes...")
        two_stage_model = TwoStageModel(
            detection_model=detection_model,
            classification_model=classification_model,
            detection_threshold=0.7  # Seuil de confiance pour la détection
        )
        
        # Sauvegarder les informations du modèle
        model_info = {
            'detection_classes': detection_class_names,
            'classification_classes': classification_class_names,
            'model_info': {
                'detection_model': {
                    'base_model': 'standard_resnet18',
                    'num_classes': len(detection_class_names)
                },
                'classification_model': {
                    'base_model': 'standard_resnet34',
                    'num_classes': len(classification_class_names)
                },
                'detection_threshold': 0.7
            },
            'date_created': time.strftime("%Y-%m-%d %H:%M:%S")
        }
        
        # Sauvegarder les informations du modèle
        model_info_path = os.path.join(MODELS_DIR, 'two_stage_model_info.json')
        with open(model_info_path, 'w') as f:
            json.dump(model_info, f, indent=4)
        
        print(f"Informations du modèle à deux étapes sauvegardées dans {model_info_path}")
        
        # Sauvegarder les informations d'entraînement dans Google Drive si nous sommes dans Colab
        if IN_COLAB:
            training_info = {
                'detection_results': detection_results if 'detection_results' in locals() else None,
                'classification_results': classification_results if 'classification_results' in locals() else None,
                'model_info': model_info,
                'training_parameters': {
                    'batch_size': BATCH_SIZE,
                    'img_size': IMG_SIZE,
                    'learning_rate': LEARNING_RATE,
                    'num_epochs': NUM_EPOCHS,
                    'early_stopping_patience': EARLY_STOPPING_PATIENCE
                },
                'date_trained': time.strftime("%Y-%m-%d %H:%M:%S")
            }
            
            training_info_path = f"{DRIVE_PROJECT_DIR}/training_info.json"
            with open(training_info_path, 'w') as f:
                json.dump(training_info, f, indent=4)
            
            print(f"Informations d'entraînement sauvegardées dans Google Drive: {training_info_path}")
        
        return two_stage_model
    except Exception as e:
        print(f"Erreur lors de la création et de la sauvegarde du modèle à deux étapes: {e}")
        return None

# Créer et sauvegarder le modèle à deux étapes si les modèles ont été entraînés
if 'detection_model' in locals() and detection_model is not None and 'classification_model' in locals() and classification_model is not None:
    two_stage_model = create_and_save_two_stage_model()
    
    if two_stage_model is not None:
        print("\nModèle à deux étapes créé et sauvegardé avec succès.")
    else:
        print("\nErreur lors de la création et de la sauvegarde du modèle à deux étapes.")
else:
    print("\nVeuillez d'abord entraîner les modèles de détection et de classification.")

## 11. Conclusion

Dans ce notebook, nous avons utilisé les fonctions existantes des modules `data_preprocessing`, `model_architecture` et `train_two_stage` pour :
1. Charger les données préparées dans le notebook précédent
2. Créer les modèles de détection et de classification
3. Entraîner et évaluer les modèles
4. Créer et sauvegarder le modèle à deux étapes

Les modèles entraînés sont sauvegardés dans Google Drive pour une utilisation ultérieure dans le notebook de test.